#### Code to extract data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src

## Imports

In [ ]:
import os
import cv2
import glob
import gdcm
import pydicom
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
from tqdm.notebook import tqdm

In [ ]:
from params import *
from data.extraction import *

In [ ]:
paths = glob.glob(TRAIN_DCM_PATH + "*/*/*.dcm")

for path in np.random.choice(paths, 1):
    print(f"Image {path}")
    img, metadata = read_xray(path)

    # sns.histplot(img.flatten())
    # plt.axvline(end, c="salmon")
    # plt.axvline(start, c="salmon")
    # plt.show()

    img_, window = auto_windowing(img.copy())
    img_, starts = remove_padding(img_)

    plt.figure(figsize=(9, 9))
    plt.imshow(img_, cmap="gray")
    plt.title(f'Window : {window}')
    plt.show()

    plt.figure(figsize=(9, 9))
    plt.imshow(img, cmap="gray")
    plt.title(f'Original')
    plt.show()
    
#     break

### Params

In [ ]:
if not os.path.exists(DCM_PATH):
    DCM_PATH = DATA_PATH

In [ ]:
SIZE = 512
DATABASE = "train"
SAVE_DIR = DATA_PATH + f"{DATABASE}_{SIZE}/"

if not os.path.exists(SAVE_DIR):
    os.mkdir(SAVE_DIR)

In [ ]:
df = pd.read_csv(DATA_PATH + "train_image_level.csv")

In [ ]:
infos = []
for i in tqdm(range(len(df))):
    study = df['StudyInstanceUID'][i]
    image = df['id'][i].split('_')[0]

    study_path = os.path.join(DCM_PATH, DATABASE, study)
    series = os.listdir(study_path)
    
    found = False
    for s in series:
        img_path = os.path.join(study_path, s, image + ".dcm")
        if os.path.exists(img_path):
            found = True
            break
            
    if not found:
        print(f'Image {i} not found')
        continue
    
    img, meta = read_xray(img_path)
    shape = img.shape

    img, window = auto_windowing(img)

    img, crop_starts = remove_padding(img)
    
    shape_crop = img.shape

    if not (i % 500):
        plt.figure(figsize=(9, 9))
        plt.imshow(img, cmap="gray")
        plt.axis(False)
        plt.show()
        
    img = cv2.resize(img, (SIZE, SIZE), interpolation=cv2.INTER_LINEAR)
    
    save_name = f"{study}_{image}.png"
    cv2.imwrite(SAVE_DIR + save_name, img)

    info_dic = {
        "study_id": [study],
        "series_id": [s],
        "image_id": [image],
        "save_name": [save_name],
        "shape": [shape],
        "shape_crop": [shape_crop],
        "window": [window],
        "crop_starts": [crop_starts],
        "photometric_interpreation": [meta.PhotometricInterpretation],
        "series_number": [meta.SeriesNumber],
        "instance_number": [meta.InstanceNumber],
    }
    infos.append(pd.DataFrame.from_dict(info_dic))

#     break

infos = pd.concat(infos).reset_index(drop=True)

if len(infos) == len(df):
    infos.to_csv(DATA_PATH + f'df_{DATABASE}_{SIZE}.csv', index=False)

In [ ]:
infos.head()